In [10]:
'''
Strategy 2: from literature: YES: working

'''

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from backtesting import Backtest, Strategy
from datetime import datetime
import talib
import joblib
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('./EURUSD_D1.csv')
data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%d %H:%M:%S')
data.set_index('Time', inplace=True)


#### Feature Engineering ####
def multivariateFeatureEngineering(data):
    
    #Trend following Indicators:

    #SMA - identofy long term trend
    data['50_sma'] = data['Close'].rolling(window=50).mean() 
    data['200_sma'] = data['Close'].rolling(window=200).mean() 

    #EMA - trend analysis: more weight applied to recent points
    data['50_ema'] = data['Close'].ewm(span=50, adjust=False).mean()
    data['100_ema'] = data['Close'].ewm(span=100, adjust=False).mean()

    #MACD
    data['12_ema'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['26_ema'] = data['Close'].ewm(span=26, adjust=False).mean()

    data['MACD_line'] = data['12_ema']-data['26_ema'] # calculate the MACD line
    data['Signal_line'] = data['MACD_line'].ewm(span=9, adjust=False).mean() # 9-preiod ema signal calculated from the Macdline

    #ADX
    # Calculate ADX using TA-Lib (14-period by default)
    data['ADX'] = talib.ADX(data['High'], data['Low'], data['Close'], timeperiod=14)

    #Momentum indicators:

    #RSI - 14-period
    data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
 
    #Stochastic Oscillator
    data['stoch_k'], data['stoch_d'] = talib.STOCH(data['High'], data['Low'], data['Close'], 
                                                fastk_period=14, slowk_period=3, slowd_period=3)

    #Volatility indicators#:

    #ATR -Default period for ATR is 14
    data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)
  
    data = data.dropna() # drop rows that have NA

    #drop certain featires
    data = data.drop(columns=['12_ema', '26_ema'])

    return data

###### Generate lag feaures #######
def multivariateFeatureLagMultiStep(data, n_past, future_steps, target_column):
    features = []
    response = []

    max_future_step = max(future_steps)
    num_features = data.shape[1]
    group_feature_lags =  1 # change grouping of lagged features

    # Adjust the loop to prevent index out of bounds
    for i in range(n_past, len(data) - max_future_step + 1):

        if group_feature_lags==1:
                
            lagged_features = []

            for feature_idx in range(num_features):
                feature_lags = data.iloc[i - n_past:i, feature_idx].values 
                lagged_features.extend(feature_lags) 

        elif group_feature_lags==0:
            features.append(data.iloc[i - n_past:i, :].values)  # Take all columns as features

        # Use .iloc for integer-based indexing and .values to get a NumPy array

        if group_feature_lags==1:
            features.append(lagged_features)

        # Extract the target values at specified future steps using .iloc
        response.append([data.iloc[i + step - 1, target_column] for step in future_steps])

    # Convert lists to NumPy arrays after the loop
    features = np.array(features)  # Shape: (num_samples, n_past, num_features)
    response = np.array(response)  # Shape: (num_samples, len(future_steps))

    features_flat = features.reshape(features.shape[0], -1)

    return features_flat, response


############################# Load saved Best model information ##################################################

best_model_info= [1, 1, ['Open', 'Low', 'sma_50', 'sma_200', 'ema_50', 'MACD_line', 'Signal_line', 'RSI', 'ATR', 'ema_100', 'High', 'Close']]

best_model_data = joblib.load('./model _weights/best_model_weights_and_scaler.pkl')
scaler = best_model_data['scaler']
weights = best_model_data['weights']
bias = best_model_data['bias']

lookback_window = best_model_info[0]
features = best_model_info[2]

data = multivariateFeatureEngineering(data) # generate additional features

#rename columns 
data = data.rename(columns={
    '50_sma': 'sma_50',
    '200_sma': 'sma_200',
    '50_ema': 'ema_50',
    '100_ema': 'ema_100'

})

################################# # Backtesting Strategy using Linear Regression #########################################

class MultiOutputForecastStrategy(Strategy):
    c = 0.00002  # Threshold based on 2 * transaction cost

    short_sl_multiplier =1.02
    short_tp_multiplier = 0.97 
    long_tp_multiplier = 1.02
    long_sl_multiplier = 0.99

    def init(self):
        self.index = 1  # Track positions and predictions
        self.test_data_subset = self.data.df[features]  # Subset of data with relevant features
        self.position_status = None  # Track whether position is long or short

    def forecast_prices(self, data_window):
        """Forecast prices for 1-day, 3-day, and 5-day horizons."""
        lagged_features = data_window.to_numpy().flatten().reshape(1, -1)  # Flatten data

        X_scaled = scaler.transform(lagged_features)  # Scale features
        predictions = np.dot(X_scaled, weights) + bias  # Forecasts (1-day, 3-day, 5-day)
        return predictions[0]  # Return predictions

    def next(self):
        if self.index >= lookback_window:
            # Get the latest data window
            data_window = self.test_data_subset.iloc[self.index - lookback_window + 1 : self.index + 1]

            # Get forecasts for 1-day, 3-day, and 5-day horizons
            forecast_1d, forecast_3d, forecast_5d = self.forecast_prices(data_window)
            current_close = self.data.Close[self.index]

            # Calculate forecasted returns
            forecasted_return_1d = forecast_1d - current_close
            forecasted_return_5d = forecast_5d - current_close

            # Prioritize short-term forecasts but confirm with long-term trends
            if forecasted_return_1d > self.c and forecasted_return_5d > self.c:
                if self.position_status != 'long':
                    print("Buy signal detected.")
                    self.position.close()  # Close any short position
                    self.buy(size=0.1, tp=self.long_tp_multiplier * current_close, sl=self.long_sl_multiplier * current_close)  # Open long position
                    self.position_status = 'long'

            elif forecasted_return_1d < -self.c and forecasted_return_5d < -self.c:
                if self.position_status != 'short':
                    print("Sell/Short signal detected.")
                    self.position.close()  # Close any long position
                    self.sell(size=0.1, tp=self.short_tp_multiplier * current_close, sl=self.short_sl_multiplier * current_close)  # Open short position
                    
                    self.position_status = 'short'

        self.index += 1

    def profit_factor_cal(self):
        trades = self.closed_trades

        gross_profit = sum(trade.pl for trade in trades if trade.pl>0)
        gross_loss = abs(sum(trade.pl for trade in trades if trade.pl <0))

        profit_fact = gross_profit/gross_loss if gross_loss !=0 else float('inf')
        
        return profit_fact
    


test_data = data.iloc[-481:]  # This assumes X_test is at the end of the dataset
bt = Backtest(test_data, MultiOutputForecastStrategy, cash =10000, commission=0, margin=.05, trade_on_close=True)
stats = bt.run()
# stats = bt.optimize(c=[0.00005, 0.0001, 0.0002, 0.0003, 0.0004], maximize='Equity Final [$]')
# stats = bt.optimize(short_tp_multiplier=[0.97,0.98, 0.99], long_tp_multiplier= [1.01, 1.02, 1.03], maximize='Equity Final [$]')
# best_params = stats._strategy._params
# print(best_params)

bt.plot() 
stats 

strategy_instance = stats._strategy  # This accesses the strategy instance
profit_factor = strategy_instance.profit_factor_cal()  # Call the function to calculate profit factor

print(profit_factor)





Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.
Sell/Short signal detected.
Buy signal detected.

## Analysis

* if lower threshold, more opportunity , high returns
    - instead of trying to capture the biggest move, identify small movement with certainty and trade based on them
    - not only that,provides more opportunity
    - higher win rate, higher sharpe ratio
* pip = 1 is the better threshold to set, maximises the returns
    
* Attempt to minimise loss, to have a higher returns
* Ultimately what also influences the returns is postion size: higher sizing == increase in returns, but consequently increase in risk
    - trade off to be made
* Advantage of this strategy is lower trade frequency >> selective on trades
    - capitalize on the best found trades
    - even if transaction costs included still profitable, because of the low trade frequency
    - able to capture upward trends easier than downward trends

* Disadvantage:
    - strategy cannot capitalize on the short trades, thus has missed opportunities
    - attempting to mitigate this results in an increase in trade frequency, and lower returns


#Best param:
    - sell sl: 2% from current price
    - sell tp: 3% from current price
    - 

#NOTE:
* No forecast or trading strategy is perfect: don't cheat by hacking them:
    >> forecast >> trade >> evaluate
